In [ ]:
# %matplotlib widget
import pickle
import numpy as np

method = "LN_COBYLA (NLopt)"
problem = "maxcut"
p = 5
n = 12
seed_pool = list(range(1000))

data = pickle.load(
    open(
        f"data/{problem}/configs/max_ar/{method}-p{p}-q{n}-s0-999.pckl",
        "rb",
    ),
)
max_ar = data["result"][:len(seed_pool),np.newaxis,np.newaxis]

data = pickle.load(
    open(
        f"data/{problem}/configs/budget/{method}-p{p}-q{n}-s{seed_pool[0]}-{seed_pool[-1]}.pckl",
        "rb",
    ),
)
config = data["config"]
result = data["result"][:,:10,:10]
initial_ar = np.array(data["initial_ar"])[:,np.newaxis,np.newaxis]

print(config, result.shape)
shots_pool = [d["shots"] for d in config["executor_kwargs"]][:10]
maxfev_pool = config["maxeval"][:10]

result = (result - initial_ar) / (max_ar - initial_ar)

In [ ]:
# print("Initial AR:", np.mean(initial_ar))
mean_ar = np.mean(result, axis=0)
indices = np.argsort(mean_ar.flat)[-1:-100:-1]
for r, c in zip(mean_ar.flatten()[indices], config.interpret(indices)):
    print(r, c)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
x, y = np.meshgrid(maxfev_pool, shots_pool, indexing='ij')
z = np.mean(result, axis=0)
surf = ax.plot_surface(x, y, z, cmap=cm.coolwarm, linewidth=0, antialiased=True)
# ax.plot_surface(x, y, np.ones_like(z) * np.mean(initial_ar), cmap=cm.coolwarm, linewidth=0, antialiased=True, rstride=1, cstride=1, edgecolor='none')
ax.set_title(method)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_ylabel('#shots/eval')
ax.set_xlabel('max #eval')
ax.set_zlabel('relative mean ar')

In [ ]:
budget = 10000

plt.figure()
x, y = np.meshgrid(maxfev_pool, shots_pool, indexing='ij')
z = np.mean(result, axis=0)
fig = plt.tricontourf(x.flat, y.flat, z.flat, levels=50)
plt.plot(maxfev_pool, budget / np.array(maxfev_pool), "ko", ms=1)
plt.colorbar(fig)

In [ ]:
xx = [11, 12, 14, 16, 20]
yy = [900, 800, 700, 600, 500]
zz = [z[0,7], z[2,6], z[4,5], z[8,4]]
zz = [z[0,8], z[1,7], z[3,6], z[5,5], z[9,4]]
print(zz)